# Chapter 6: Word Vectors

<b>...</b> : là những đoạn chưa/không dịch

# Nội dung chương:
* Hiểu word vectors được tạo như thế nào
* Sử dụng model đã train trước (pretrained model) cho ứng dụng của bạn
* Suy luận với word vectors để giải bài toán thực tế
* Mô phỏng word vectors
* Khám phá một số cách sử dụng đáng kinh ngạc cho word embedding

Một trong những bước phát triển gần đây của NLP là khám phá ra word vector. Chương này sẽ giúp bạn hiểu nó là gì và sử dụng chúng như thế nào. Bạn sẽ học làm thế nào để lấy ra được 1 số mối quan hệ ẩn giữa các từ mà các chương trước không làm được. 

Trong các chương trước, chúng tôi không biết đến context của một từ. Chúng tôi không biết các từ xung quanh mỗi tứ. Chúng tôi không biết đến hàng xóm của một từ có ý nghĩa và mối quan hệ như thế nào cho nghĩa của một câu. BoW của chúng ta như một mớ lộn xọng tất cả các từ trong mỗi documen thành một bag. Trong chương này, bạn sẽ tạo nhiều BoW nhỏ hơn từ một hàng xóm của chỉ một số từ, thường ít hơn 10 token. Bạn sẽ đảm bảo rằng hàng xóm có nghĩa không tràn qua câu liền kề. Chương này sẽ tập trung giúp bạn traning word vector trên các từ thích hợp.

Vector từ mới của chúng ta sẽ có thể xác định từ đồng nghĩa, từ trái nghĩa, hoặc từ mà chỉ thuộc một loại tương tự, như là người, động vật, địa điểm, dự án, tên hoặc hợp đồng. Chúng tôi có thể làm điều này trước, với LSA trong chương 4, nhưng bạn giới hạn chặt chẽ hơn trên hàng xóm của một từ sẽ phản ảnh chính xác chặt chẽ của vector từ. LSA của từ, n-grams, và các document không mang hình ảnh của tất cả ý nghĩa của một từ, một từ có nhiều ý nghĩa ngụ ý hay ẩn. Một số ý nghĩa của từ bị mất với LSA.

<b>WORD VECTOR</b>: word vector là số hoá vector từ, hoặc ý nghĩa, bao gồm cả nghĩa đen và nghĩa bóng của nó. Vì vậy word vectors có thể mang hình ảnh ý nghĩa của từ, giống như "peopleness", "animalness", "placeness", "thingness", "conceptness". Và chúng tổ hợp tất cả thành một vector dense (no zeros) mang các giá trị floating. vector dense này cho phép truy vấn hoặc lập luận logic.

## 6.1 Secmactic queries and analogies (Truy vẫn ngữ nghĩa và logic)
Tốt, các word vector tốt là gì? Bạn đã bao giờ cố gắng để gọi một tên người nổ tiếng nhưng bạn chỉ có một ấn tượng chung về chúng, giống như sau:

`She invented something to do with physics in Europe in the early 20th century`

Nghĩa là:

`Cô ấy sáng chế ra một số thứ trong lĩnh vực vật lý ở châu Âu đầu thế kỷ 20`

Nếu bạn truy cập câu này vào Google hoặc Bing, bạn có thể không có câu trả lời cho cái bạn cần tìm, đó là "Marie Curie". Goole Search sẽ chỉ cho bạn đường dẫn tới các nhà vật lý nổi tiếng, cả nam và nữ. Bạn phải lướt nhiều trang để tìm câu trả lời cho bạn. Nhưng một lần bạn đã tìm "Marie Curie". Google hoặc Bing sẽ note điều này. Chúng sẽ trả về những kết quả tốt hơn bạn tìm cho những lần sau cho một nhà khoa học.

Với word vectors, bạn có thể tìm kiếm các từ hoặc tên cái mà tổ hợp từ các ý nghĩa của từ "woman", "Europe","physics", "scientist" và "famous". và sẽ giúp bạn có được token "Marie Curie" mà bạn đang tìm. Và tất cả bạn làm được điều này là thêm word vector cho mỗi word bạn muốn tổ hợp.

`answer_vector = wv['woman'] + wv['Europe'] + wv['physics'] + wv['scientist'] + ... `

Trong chương này, chúng tôi sẽ cho bạn thấy cách chính xác để làm truy vấn này. Và chúng tôi thậm chí chỉ cho bạn cách trừ bias giới tínhnhư thế nào từ word vector tổ hợp để có câu trả lời của bạn:

`answer_vector = wv['woman'] + wv['Europe'] + wv['physics'] + ... + wv['scientist'] - wv['male'] - 2*wv['man']`

Với word vectors, bạn có thể đưa ra từ "man" trái với "women"!

### 6.1.1 Câu hỏi tương tự
Điều gì nếu bạn muốn nói lại câu hỏi của bạn như một câu hỏi tương tự? Điều gì nếu bạn truy vấn một câu hỏi giống như này:

`Thắng đẹp trai hay xấu trai`

Đây là một câu hỏi dạng trả lời A, B, C, D. Bạn có thể so sánh 2 vector từ:

`wv['Thắng'] + wv['đẹp_trai'] - wv['xấu_trai']` và

`wv['Thắng'] - wv['đẹp_trai'] + wv['xấu_trai']`

xem cái nào cho giá trị tốt hơn thì đó là câu trả lời.

Word vector có thể trả lời các vấn đề mơ hồ hoặc câu hỏi tương tự. Word vector có thể giúp abjn nhớ mỗi từ hoặc tên của một ai đó, miễn là vector từ đó tồn tại trong tập từ vựng (vocabulary) của bạn. Và word vector làm việc tốt cho các hỏi mà bạn không thể có được từ form tìm kiếm hay tương tự. Bạn có thể học về một số toán học liên quan cho word vector trong section 6.2.1

## 6.2 Word vectors
Trong năm 2012, Thomas Mikolov, một thực tập sinh tại Microsoft, tìm một phương pháp mã hoá ý nghĩa của từ trong không gian vector nhỏ nhất. Mikolov trained một mạng neural để dự đoán từ xung quanh gần mỗi từ mục tiêu nào đó. Trong năm 2013, một lần ở Google, Mikolov và team của anh ấy đã phát hành sản phẩm tạo các vector từ và gọi là word2vec.

Word2Vec học nghĩa của từ chỉ đơn thuần là xử lý một corpus lớn các văn bản không nhãn. Không thứ gì có nhãn của từ trong bộ từ vựng Word2Vec. Không ai nói cho thuật toán Word2Vec rằng Marie Curie là một nhà khoa học, Timbers là một đội bóng hay Seattle là một thành phố, hoặc Portland là một thành phố trong cả Oregon và Maine. Và không một nói với word2vec là soccer là sport, hoặc một team là một nhóm người, một thành phố là một địa điểm. Word2Vec có thể học nhiều hơn! Tất cả những gì bạn cần là một corpus lớn đủ để đề cập đến rằng Marie Curie và Timbers và Portland cần các từ khác liên kết với science hoặc soccer hoặc cities.

Thuật toán học không giám sát tự nhiên của Word2Vec là hữu ích. Các từ hoàn toàn không gán nhã, không được phân loại, không có cấu trúc.

Học không giám sát và học giám sát là 2 phương pháp học khác nhau trong Machine learning.

Thay vì cố gắng train một mạng neural học trực tiếp ý nghĩa của các từ mục tiêu (dựa trên gán nhãn ý nghĩa), bạn dạy mạng dự đoán từ gần với từ mục tiêu trong câu. Vậy có nhĩa là bạn làm không có nhãn: Các từ gần bạn cố gắng dự đoán. Nhưng bởi vì các nhãn đến từ bản thân dataset và đòi hỏi không gán nhãn, thuật toán Word2Vec traning rõ ràng là thuật toán học không giám sát.

Một số lĩnh vực khác mà công nghệ học không giám sát được sử dụng như là trong mô hình time series. Time series thường được trained để dự đoán giá trị tiếp theo trong chuỗi cơ bản trên window của các giá trị trước. Bài toán time series khá giống bài toán xử lý ngôn ngữ tự nhiên trong nhiều cách khác nhau, bởi vì chúng giải quyết với chuỗi các giá trị đã sắp xếp (word hoặc số).

Word2Vec sẽ học mối liên kết giữa các từ mà chính bạn có thể không biết. Bạn có biết rằng mỗi từ có vị trí, ngữ nghĩa và liên kết với nó? Nếu bất kỳ từ nào đó trong corpus đặc biệt, như là "placeness", "peoplesness" ... , tất cả các từ khác cũng sẽ nhận được một số điểm từ những đặc tính trong word vector của bạn. Ý nghĩa của một từ "rubs off" trên các từ hàng xóm khi Word2Vec học word vector.

Tất cả các từ trong corpus sẽ biểu diễn bởi vector số, giống như word-topic vector trong chương 4. Trong LSA, các từ chỉ xuất hiện trong các cùng document. Với Word2Vec, các từ phải xuất hiện gần các từ khác trong 1 câu. Và trọng số vector topic của word2vec có thể thêm hoặc trừ đi vector mới mà có ý nghĩa gì đó.

Một mô hình thông minh giúp bạn hiểu word vector là thế nào. Mỗi trọng số hoặc điểm có số chiều xác định ý nghĩa của từ.

...

**Không có gì đặc sắc**

Trong chương này, chúng tôi muốn cải thiện về cơ quan đại diện vector LSA từ chúng tôi giới thiệu trong chương trước. vector chủ đề xây dựng từ toàn bộ tài liệu sử dụng LSA là tuyệt vời cho phân loại tài liệu, tìm kiếm ngữ nghĩa, và clustering. Nhưng vector chủ đề chữ rằng LSA sản xuất không đủ chính xác để được sử dụng cho ngữ nghĩa lập luận hoặc phân loại và phân nhóm các cụm từ ngắn hoặc từ ghép. bạn sẽ sớm tìm hiểu làm thế nào để đào tạo một lớp mạng thần kinh cần thiết để sản xuất các chính xác hơn và thú vị hơn vectơ từ. Và bạn sẽ thấy lý do tại sao họ đã thay thế LSA vectơ word-chủ đề cho nhiều ứng dụng liên quan đến tài liệu ngắn hoặc báo cáo.

### 6.2.1 Vector-oriented reasoning (Lập luận vector theo hướng)
Word2vec lần đầu được công bố năm 2013 tại hội nghị ACL. Word2Vec embeding chính xác hơn (45%) 4 lần so với mô hình LSA (11%) với vấn đề trả lời câu hỏi tương tự như nói ở trên. Độ chính xác cải tiến đáng ngạc nhiên, trong thực tế, báo cáo ban đầu của Mikolov bị loại bởi International Conference trên Learning Representations. Nhà phê bình cho rằng hoạt động của mô hình quá tốt đến mức khó tin. Phải mất gần 1 năm cho team của Mikolov để được chấp nhận source code cho Computational Linguistics.

Ngẫu nhiên, với word vectors, câu hỏi giống như là:

`Portland Timbers + Seattle - Portland = ? `

có thể giải quyết bằng đại số vector.

<img src="./images/w2vmath.png"/><br/>

Mô hình Word2Vec bao gồm thông tin về mối quan hệ giữa các từ, bao gồm sự tương đồng. Mô hình Word2Vec biết rằng *portland* và *portland timbers* xấp xỉ khoảng cách của *Seattle* và *Seattle Sounders*. Và các khoảng cách này (khoảng cách giữa cặp vector) là phải cùng phương. Vì vậy mô hình Word2Vec có thể sử dụng chúng để trả lời cho câu hỏi đội thể thao của bạn. Bạn có thể thêm phần chênh lệch giữa 2 vector *Portland* và *Seattle* cho vecot *Porland Timber*, Mà nên giúp bạn có ???c ?óng vào vector cho thuật ngữ
Seattle Sounders:

... 

**Đoạn này xàm lồng mode**

### 6.2.2 Biểu diễn Word2Vec tính toán như thế nào?
Word vector đại diện cho nghĩa của từ trong context mà nó được đào tạo. Cho phép bạn không chỉ trả lời câu hỏi analogy (tương tự) mà còn biết về nghĩa của các từ tổng quát hơn với vector đại số. Nhưng làm thế nào để tính toán vector đại diện đó? Có 2 cách có thể train Word2Vec embedding:
* *skip-gram* dự đoán context của từ (các từ output) từ một từ input.
* *continouous bag-of-words (CBOW)* dự đoán từ mục tiêu (từ output) từ các từ gần nó (input words). Chúng tôi chỉ cho bạn như thế nào và khi nào sử dụng mô hình nào để train Word2Vec trong section sắp tới.

Sau chương này, chúng tôi giới thiệu cho bạn cách sử dụng các mô hình pretrained từ khác, như là Glove và FastText.

#### SKIP-GRAM APPROACH
Ở trong mô hình skip-gram, bạn cố gắng dự đoán các từ xung quanh một từ input. Trong câu về Monet, theo dõi ví dụ của chúng tôi, "painted" là training input của mạng neural. Tương ứng oupt ví dụ skip-gram như hình dưới. Dự đoán các từ của skip-gram là các từ hàng xóm "Claude", "Monet", "the", và "Grand".

<img src="./images/skipgram.png"/>

#### WHAT IS SKIP-GRAM?
Skip-gram là n-grams bao gồm gaps (khoảng trống) bởi vì bạn bỏ qua các token can thiệp. Trong ví dụ, bạn dự đoán "Clause" từ input token "painted", và bạn bỏ qua token "Monet".

Cấu trúc của mạng neural sử dụng để predict các từ giống như bạn đã học ở chương 5. Nhưng bạn có thể thấy hình 6.4, bạn bao gồm 2 layers của weights, tại đây hidden layer bao gồm *n* neurons; n là số chiều vector sử dụng để biểu diễn một từ. Cả input và oupt layer bao gồm M Neurons, tại đây M là số các số trong từ vựng. Output layer hàm kích hoạt là hàm softmax, thường được sử dụng cho các bài toán phân lớp.

<img src="./images/skipgrammodel.png"/>

#### WHAT IS SOFTMAX?
...

**Không có gì đặc sắc**

#### LÀM THẾ NÀO MẠNG HỌC ĐƯỢC BIỂU DIỄN VECTOR?
Để train một mô hình Word2Vec, bạn sử dụng kiến thức chương 2. Cho ví dụ, trong bảng dưới đây, $w_t$ biểu diễn dạng one-hot vector cho token tại vị trí t. Vì vậy nếu bạn muốn train một hình Word2Vec skip-gram sử dụng một window size cho 2 từ, bạn xem xét 2 từ trước và sau mỗi từ mục tiêu. Sau đó bạn sẽ sử dụng 5-gram tokenizer từ chương 2 turn một câu như này:

`sentence = "Claude Monet painted the Grand Canal of Venice in 1806."`

thành 10 5-gram với input word tại trung tâm.

Bảng sau biểu diexn 10 5-gram cho câu về Monet:

<img src="./images/tablemonet.png"/>

Tập training gồm từ input và các từ xung quanh nó.Trong trường hợp bốn từ xung quanh, bạn sẽ sử dụng bốn lần lặp đào tạo, trong đó mỗi chữ đầu ra đã được dự đoán dựa vào từ đầu vào.

Mỗi từ sẽ được biểu diễn dạng vector one-hot trước khi chúng được đưa vào mạng neural.

Sau khi huấn luyện của mạng lưới thần kinh hoàn tất, bạn sẽ nhận thấy rằng các trọng có được huấn luyện để đại diện cho ý nghĩa ngữ nghĩa. Nhờ các vector một nóng chuyển đổi các thẻ của bạn, mỗi hàng trong ma trận trọng lượng đại diện cho mỗi từ từ từ vựng cho corpus của bạn. Sau khi đào tạo, ngữ nghĩa các từ tương tự sẽ có vectơ tương tự, bởi vì họ được đào tạo để dự đoán những lời xung quanh tương tự. Đây là hoàn toàn huyền diệu!


#### Biểu diễn đại số tuyến tính
...

<img src="./images/skipgrammatrix.png"/>

#### CBOW

<img src="./images/cbowmodel.png"/>

#### Khi nào sử dụng Skip-gram vs CBOW

...

### 6.2.3 Thực hành với model gensim.word2vec
...
### 6.2.4 Tạo ra biểu diễn vector từ của riêng bạn
Trong một số trường hợp, bạn có thể muốn tạo mô hình vector từ miền cụ thể của riêng bạn.
Làm như vậy có thể cải thiện tính chính xác của mô hình của bạn nếu đường ống NLP bạn đang xử lý
tài liệu có sử dụng các từ trong một cách mà bạn sẽ không tìm thấy trên Google News trước khi
Năm 2006, khi Mikolov đào tạo các tài liệu tham khảo mô hình Word2vec. Hãy nhớ, bạn cần
nhiều tài liệu để làm điều này cũng như Google và Mikolov đã làm. Nhưng nếu lời của bạn là
đặc biệt hiếm trên Google News, hoặc văn bản của bạn sử dụng chúng theo những cách độc đáo trong một
miền hạn chế, chẳng hạn như các văn bản y tế hoặc bảng điểm, một mô hình từ miền cụ thể
có thể cải thiện độ chính xác mô hình của bạn. Trong phần sau, chúng tôi chỉ cho bạn cách để đào tạo
mô hình Word2vec của riêng bạn.

...

In [3]:
import multiprocessing
multiprocessing.cpu_count()

8

In [6]:
# input là list token các câu, các từ.
num_feattures = 300 # số phần tử vector từ để biểu diễn từ
min_word_count = 3 # số min cho phép trong mô hình W2V
num_workers = 2 # số nhân CPU sử dụng để train
windown_size = 6 # cửa số từ , ngữ cảnh
subsampling = 1e-3 # tỷ lệ mẫu con

In [ ]:
model = Word2Vec(
... token_list,
... workers=num_workers,
... size=num_features,
... min_count=min_word_count,
... window=window_size,
... sample=subsampling)

In [ ]:
# loại bỏ các trọng số không cần thiết
model.init_sims(replace=True)

In [ ]:
# lưu model
model_name = "my_domain_specific_word2vec_model"
model.save(model_name)

In [ ]:
# Test
from gensim.models.word2vec import Word2Vec
model_name = "my_domain_specific_word2vec_model"
model = Word2Vec.load(model_name)
model.most_similar('radiology')

### 6.2.5 Word2vec vs Glove
...
### 6.2.6 FastText
...
### 6.2.7 Word2Vec vs LSA
...
### 6.2.8 Biểu diễn quan hệ từ
...

Nội dung chính:

* Khoảng cách giữa 2 vector từ
* Cosin similarity giữa 2 vector từ
* PCA giảm chiều và biểu diễn quan hệ từ trên đồ thị

### 6.2.9 Unnatural words
...
### 6.2.10 Document similarity với Doc2Vec
Khái niệm Word2Vec có thể mở rộng cho câu, đoạn hoặc document. Ý tưởng tự đoán từ tiếp theo dựa trên các từ trước có thể mở rộng bằng cách training một đoạn hoặc vector document. (Như hình dưới). Trong trường hợp này, dự đoán không chỉ xem xét các từ trước, mà còn biểu diễn vector đoạn văn hoặc document. Nó có thể xem xét như một input word thêm vào để dự đoán. Qua nhiều lần, thuật toán học một document hoặc biểu diễn một đoạn văn từ tập training.

<img src="./images/doc2vec.png"/>

Vậy làm thế nào để vector document sinh ra từ các document sau khi qua các giai đoạn training? Trong giai đoạn suy luận, thuật toán thêm nhiều vector document và ma trận document và tính toán và thêm vector cơ sở trên ma trận word vector frozen, và các trọng số. Suy ra một vector document, bạn có thể tạo ngay một biểu diễn ngữ nghĩa của document.

Mở rộng khái niệm Word2Vec với thêm một vector document hoặc đoạn văn sử dụng dự đoán từ, bạn có thể sử dụng ngay vector document đã train cho các tác vụ khác nhau, như là tìm kiếm document tương đồng trong một corpus.

#### HOW TO TRAIN DOCUMENT VECTORS
Giống như train word vector, bạn sử dụng packet gensem để train document vector theo dõi ví dụ sau:

In [5]:
import multiprocessing
num_cores = multiprocessing.cpu_count()
# Mô hình gensim Doc2Vec bao gồm vector word cũng như vector document cho mỗi
# document trong corpus của bạn
from gensim.models.doc2vec import TaggedDocument, Doc2Vec
# simple_preprocess là xử lý bỏ qua ký hiệu và dấu chấm câu ...
from gensim.utils import simple_preprocess

corpus=['thắng đẹp trai đây là văn bản thứ nhất', \
        'đây là văn bản thứ hai của thắng']
training_corpus = []
for i, text in enumerate(corpus):
    tagged_doc = TaggedDocument(simple_preprocess(text),[i])
    training_corpus.append(tagged_doc)
    
model = Doc2Vec(size=100, min_count=2, workers=num_cores, iter=10)
# trước khi mô hình train, cần phải complie vocabulary
model.build_vocab(training_corpus)
model.train(training_corpus, total_examples=model.corpus_count, epochs=model.iter)

c:\program files\python37\lib\site-packages\ipykernel_launcher.py:19: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).


Sau khi mô hình Doc2Vec được train, bạn có thể suy ra vector document mới:

In [6]:
model.infer_vector(simple_preprocess('đây là document không có'), steps=10)

array([-0.0038117 , -0.00337257,  0.00179766, -0.00388346,  0.00105044,
       -0.00117316,  0.00206711,  0.00260973,  0.0047255 ,  0.00310806,
       -0.00132465,  0.00209348,  0.00017028,  0.00470623,  0.0036723 ,
        0.00409537,  0.00433532,  0.00107362, -0.00269005, -0.00386435,
        0.00061761,  0.00123246, -0.00428007,  0.00375596,  0.00354742,
       -0.0045937 , -0.00101515,  0.00075844,  0.00223326, -0.00212732,
        0.00317319, -0.00100764, -0.00139516, -0.0002363 , -0.00092861,
       -0.00017096,  0.00330099,  0.00297676, -0.00345007, -0.00119124,
        0.00402085, -0.00196332, -0.0044789 , -0.00115892,  0.0002236 ,
       -0.00048676,  0.00258138, -0.00419255, -0.00029039,  0.00328402,
        0.00335046,  0.00047524, -0.00463963, -0.00299459,  0.00243686,
       -0.00066489, -0.00028533, -0.00010936, -0.00046491,  0.0022787 ,
        0.00084644, -0.00439386, -0.00298537, -0.00348642,  0.00353694,
        0.003474  , -0.00099111, -0.00318219, -0.00499911, -0.00

Doc2Vec đòi hỏi training trước khi suy ra vector mới. Trên ví dụ, bạn cập nhật vector đã train qua 10 step.

Với 1 số bước step, bạn có thể nhanh chóng train toàn bộ corpus của document và tìm các document tương đồng. Bạn có thể sinh một vector cho mỗi document trong corpus của bạn và rồi tính toán khoảng cách cosin giữa mỗi vector document. Ngoài ra, có thể phân nhóm vector docuemtn của corpus với k-means.